In [1]:
import sys
import os
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import warnings
warnings.filterwarnings("ignore")

# Get the absolute path to the src directory
sys.path.append(os.path.abspath("../src"))

from train_lora_model import train_lora_model


In [2]:
import h5py

with h5py.File("../lotka_volterra_data.h5", 'r') as f:
    print(list(f.keys()))
    # If there's only one key, try this:
    data = f[list(f.keys())[0]]
    print(data.dtype)


['time', 'trajectories']
float64


In [3]:
trained_model, val_loss = train_lora_model(
    data_path="../lotka_volterra_data.h5",
    lora_rank=4,
    context_length=512,
    learning_rate=1e-5,
    batch_size=4,
    max_steps=1000,     
    input_fraction=0.7,
    verbose=True
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Using scale factor: 0.326
Using scale factor: 0.363
Using scale factor: 0.340
Using scale factor: 0.250
Using scale factor: 0.716
Using scale factor: 0.195
Using scale factor: 0.408
Using scale factor: 0.159
Using scale factor: 0.113
Using scale factor: 0.183
Using scale factor: 0.232
Using scale factor: 0.418
Using scale factor: 0.117
Using scale factor: 0.128
Using scale factor: 0.182
Using scale factor: 0.162
Using scale factor: 0.403
Using scale factor: 0.311
Using scale factor: 0.139
Using scale factor: 0.382
Using scale factor: 0.297
Using scale factor: 0.127
Using scale factor: 0.166
Using scale factor: 0.134
Using scale factor: 0.343
Using scale factor: 0.337
Using scale factor: 0.120
Using scale factor: 0.105
Using scale factor: 0.315
Using scale factor: 0.375
Using scale factor: 0.269
Using scale factor: 1.039
Using scale factor: 0.135
Using scale factor: 0.169
Using scale factor: 0.446
Using scale factor: 0.267
Using scale factor: 0.298
Using scale factor: 0.284
Using scale 

LoRA Training:   0%|          | 0/1000 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Step 1000/1000:   0%|          | 0/1000 [23:22<?, ?it/s, loss=0.675]

✅ Training complete. Running validation...


Step 1000/1000:   0%|          | 0/1000 [24:21<?, ?it/s, loss=0.675]

📊 Average validation loss: 0.8813


In [4]:
print(f"\n📉 Final validation loss after training: {val_loss:.4f}")
print("🧠 Model trained with:")
print(f" - LoRA rank: 4")
print(f" - Context length: 512 tokens")
print(f" - Learning rate: 1e-5")
print(f" - Batch size: 4")
print(f" - Training steps: 1000")
print(f" - Training set: 70% of trajectories")
print(f" - Validation set: 30% of trajectories (not seen during training)")



📉 Final validation loss after training: 0.8813
🧠 Model trained with:
 - LoRA rank: 4
 - Context length: 512 tokens
 - Learning rate: 1e-5
 - Batch size: 4
 - Training steps: 1000
 - Training set: 70% of trajectories
 - Validation set: 30% of trajectories (not seen during training)


In [ ]:
from load_qwen import load_qwen_model
from evaluate_model import evaluate_model
from preprocessor import LLMTIMEPreprocessor
import h5py

# Load val series from file
with h5py.File("../lotka_volterra_data.h5", "r") as f:
    trajectories = f["trajectories"][:]
    prey_val = trajectories[700:1000, :, 0]
    predator_val = trajectories[700:1000, :, 1]

# Format val texts
preprocessor = LLMTIMEPreprocessor()
val_texts = []
for i in range(300):
    text, _, _ = preprocessor.preprocess_sample(prey_val[i], predator_val[i])
    val_texts.append(text)

# Untrained model
untrained_model, tokenizer = load_qwen_model()
untrained_loss = evaluate_model(untrained_model, val_texts, tokenizer)



print(f"\n📊 Untrained Val Loss: {untrained_loss:.4f}")



ModuleNotFoundError: No module named 'evaluate_model'